In [1]:
import pandas as pd
import math
import collections
import datetime


# 국가, 날짜별 치사율 데이터 가져오기

#### 데이터 소스: https://github.com/owid/covid-19-data/tree/master/public/data

In [3]:
data = pd.read_csv("../../data/owid-covid-data.csv")
data

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17420,NaN,International,2020-02-28,705,0,4,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17421,NaN,International,2020-02-29,705,0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17422,NaN,International,2020-03-01,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17423,NaN,International,2020-03-02,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 국가 filtering하기

In [4]:
countrySeries = data.iso_code
countryList = list(set(countrySeries)) # set으로 만들어서 중복 다 없앰.

len(countryList)

212

In [5]:

caseData = data.total_cases

In [6]:
selectedCountries = []
countryRowList = []
for country in countryList:
    check = False
    for i in range(len(countrySeries)):
        if(country == countrySeries[i]):
            check = True
        if(check and country != countrySeries[i]): # 맨 마지막 total값을 가져오기 위해서 country[i]이름이 달라지면 i-1을 리스트에 넣는다.
            if(caseData[i-1] > 3000 and country != 'OWID_WRL'): # case가 3000명이 넘는 것만 들고 옴, OWID_WRL은 전세계 데이터라 뺌
                countryRowList.append(i-1)
                selectedCountries.append(country)
            break
            
len(countryRowList) 

68

In [7]:
selectedCountries

['TUR',
 'CHL',
 'GHA',
 'CAN',
 'BGD',
 'MAR',
 'LUX',
 'COL',
 'IRN',
 'RUS',
 'AFG',
 'ROU',
 'MDA',
 'BLR',
 'ARM',
 'ARG',
 'IND',
 'KAZ',
 'PAN',
 'EGY',
 'AUS',
 'MEX',
 'KWT',
 'DNK',
 'QAT',
 'ZAF',
 'HUN',
 'BHR',
 'DOM',
 'IRQ',
 'BEL',
 'CZE',
 'AUT',
 'PHL',
 'IDN',
 'DEU',
 'BRA',
 'CHN',
 'NOR',
 'SRB',
 'JPN',
 'USA',
 'OMN',
 'ARE',
 'KOR',
 'FIN',
 'SGP',
 'THA',
 'IRL',
 'DZA',
 'MYS',
 'BOL',
 'POL',
 'SAU',
 'NLD',
 'ISR',
 'FRA',
 'NGA',
 'ITA',
 'PER',
 'ECU',
 'PAK',
 'PRT',
 'UKR',
 'SWE',
 'ESP',
 'GBR',
 'CHE']

### 일별 치사율 구하기

In [9]:

data = data[["iso_code",'date', 'new_cases', 'new_deaths']] 
fatalityDataList = data.values.tolist()

In [10]:
fatalityDataList

[['ABW', '2020-03-13', 2, 0],
 ['ABW', '2020-03-20', 2, 0],
 ['ABW', '2020-03-24', 8, 0],
 ['ABW', '2020-03-25', 5, 0],
 ['ABW', '2020-03-26', 2, 0],
 ['ABW', '2020-03-27', 9, 0],
 ['ABW', '2020-03-28', 0, 0],
 ['ABW', '2020-03-29', 0, 0],
 ['ABW', '2020-03-30', 22, 0],
 ['ABW', '2020-04-01', 5, 0],
 ['ABW', '2020-04-02', 0, 0],
 ['ABW', '2020-04-03', 5, 0],
 ['ABW', '2020-04-04', 2, 0],
 ['ABW', '2020-04-05', 2, 0],
 ['ABW', '2020-04-06', 0, 0],
 ['ABW', '2020-04-07', 7, 0],
 ['ABW', '2020-04-08', 3, 0],
 ['ABW', '2020-04-09', 3, 0],
 ['ABW', '2020-04-10', 5, 0],
 ['ABW', '2020-04-11', 4, 0],
 ['ABW', '2020-04-12', 6, 0],
 ['ABW', '2020-04-13', 0, 0],
 ['ABW', '2020-04-14', 0, 0],
 ['ABW', '2020-04-15', 0, 0],
 ['ABW', '2020-04-16', 1, 1],
 ['ABW', '2020-04-17', 2, 0],
 ['ABW', '2020-04-18', 1, 1],
 ['ABW', '2020-04-19', 0, 0],
 ['ABW', '2020-04-20', 1, 0],
 ['ABW', '2020-04-21', 0, 0],
 ['ABW', '2020-04-22', 0, 0],
 ['ABW', '2020-04-23', 3, 0],
 ['ABW', '2020-04-24', 0, 0],
 ['ABW', 

# Factor 데이터 time-series로 바꾸기

#### 데이터 소스: https://www.kaggle.com/vishalvjoseph/weather-dataset-for-covid19-predictions

In [137]:
data = pd.read_csv("../new_weather_Processed_Data.csv")
data=data.values.tolist()
data

[['AFG',
  '2019-12-31',
  36060.0,
  0.0156,
  1019.1,
  2.56,
  372.6,
  40.24500000000001,
  0.6],
 ['AFG', '2020-01-01', 36120.0, 0.0235, 1021.2, 2.06, 330.2, 34.66, 0.9],
 ['AFG', '2020-01-02', 36120.0, 0.0016, 1022.7, 2.45, 320.3, 37.665, 0.76],
 ['AFG',
  '2020-01-03',
  36120.0,
  0.0003,
  1021.9,
  2.95,
  309.6,
  39.30500000000001,
  0.69],
 ['AFG', '2020-01-04', 36180.0, 0.0145, 1016.1, 1.98, 308.2, 39.045, 0.88],
 ['AFG', '2020-01-05', 36240.0, 0.0196, 1016.7, 2.6, 323.1, 36.32, 0.85],
 ['AFG', '2020-01-06', 36240.0, 0.0004, 1022.7, 4.28, 331.1, 33.905, 0.58],
 ['AFG', '2020-01-07', 36300.0, 0.0, 1025.3, 5.06, 319.7, 35.67, 0.41],
 ['AFG', '2020-01-08', 36360.0, 0.0, 1023.1, 3.56, 322.0, 36.305, 0.31],
 ['AFG',
  '2020-01-09',
  36420.0,
  0.0001,
  1023.4,
  3.79,
  317.2,
  37.99500000000001,
  0.34],
 ['AFG', '2020-01-10', 36480.0, 0.0142, 1019.9, 2.8, 313.1, 35.83, 0.76],
 ['AFG', '2020-01-11', 36540.0, 0.0454, 1013.7, 2.76, 320.7, 31.375, 0.89],
 ['AFG', '2020-01-12'

In [138]:
newData= []
for row in data:
    if(row[0] in selectedCountries):
        for fRow in fatalityDataList:
            if(fRow[0] == row[0] and fRow[1] == row[1]):
                row.append(fRow[2])
                row.append(fRow[3])
                newData.append(row)
data = newData
len(data)


6121

In [139]:
data

[['AFG',
  '2019-12-31',
  36060.0,
  0.0156,
  1019.1,
  2.56,
  372.6,
  40.24500000000001,
  0.6,
  0,
  0],
 ['AFG', '2020-01-01', 36120.0, 0.0235, 1021.2, 2.06, 330.2, 34.66, 0.9, 0, 0],
 ['AFG',
  '2020-01-02',
  36120.0,
  0.0016,
  1022.7,
  2.45,
  320.3,
  37.665,
  0.76,
  0,
  0],
 ['AFG',
  '2020-01-03',
  36120.0,
  0.0003,
  1021.9,
  2.95,
  309.6,
  39.30500000000001,
  0.69,
  0,
  0],
 ['AFG',
  '2020-01-04',
  36180.0,
  0.0145,
  1016.1,
  1.98,
  308.2,
  39.045,
  0.88,
  0,
  0],
 ['AFG', '2020-01-05', 36240.0, 0.0196, 1016.7, 2.6, 323.1, 36.32, 0.85, 0, 0],
 ['AFG',
  '2020-01-06',
  36240.0,
  0.0004,
  1022.7,
  4.28,
  331.1,
  33.905,
  0.58,
  0,
  0],
 ['AFG', '2020-01-07', 36300.0, 0.0, 1025.3, 5.06, 319.7, 35.67, 0.41, 0, 0],
 ['AFG', '2020-01-08', 36360.0, 0.0, 1023.1, 3.56, 322.0, 36.305, 0.31, 0, 0],
 ['AFG',
  '2020-01-09',
  36420.0,
  0.0001,
  1023.4,
  3.79,
  317.2,
  37.99500000000001,
  0.34,
  0,
  0],
 ['AFG', '2020-01-10', 36480.0, 0.0142,

### 각 factor 최대 최소 구하기

In [145]:
minValue = [9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999]
maxValue = [-9999999999,-9999999999,-9999999999,-9999999999,-9999999999,-9999999999,-9999999999]

In [146]:
for row in data:
    for j in range(2,9,1):
        if(not math.isnan(row[j])):
            if(float(row[j]) < minValue[j-2]):
                minValue[j-2] = float(row[j])
            if(float(row[j]) >maxValue[j-2]):
                maxValue[j-2] = float(row[j])

In [147]:
print(minValue)
print(maxValue)

[16020.0, 0.0, 952.4, 1.3, 216.1, -18.19, 0.04]
[56700.0, 0.1919, 1047.2999999999997, 31.52, 481.8, 98.205, 1.0]


### factor당 40개로 구간을 나눠 날짜, 국가 채우기

In [148]:
valuePartition = []
for i in range(len(minValue)):
    eachValue = []
    for j in range(40):
        partition = minValue[i]+ j*(maxValue[i] - minValue[i])/40.0
        eachValue.append(partition)
    eachValue.append(maxValue[i])
    valuePartition.append(eachValue)

In [149]:
dataPartition = []
for i in range(7):
    dataPartition.append([])
    for j in range(40):
        dataPartition[i].append([])

In [151]:
for row in data:
    for index in range(2,9,1):
        if(not math.isnan(row[index])):
            for j in range(40):
                if(valuePartition[index-2][j] <= float(row[index])<valuePartition[index-2][j+1]):
                    dataPartition[index-2][j].append([row[0], row[1], row[9], row[10]])
                elif(valuePartition[index-2][40] <= float(row[index])):
                    dataPartition[index-2][39].append([row[0], row[1], row[9], row[10]])

In [152]:
valuePartition

[[16020.0,
  17037.0,
  18054.0,
  19071.0,
  20088.0,
  21105.0,
  22122.0,
  23139.0,
  24156.0,
  25173.0,
  26190.0,
  27207.0,
  28224.0,
  29241.0,
  30258.0,
  31275.0,
  32292.0,
  33309.0,
  34326.0,
  35343.0,
  36360.0,
  37377.0,
  38394.0,
  39411.0,
  40428.0,
  41445.0,
  42462.0,
  43479.0,
  44496.0,
  45513.0,
  46530.0,
  47547.0,
  48564.0,
  49581.0,
  50598.0,
  51615.0,
  52632.0,
  53649.0,
  54666.0,
  55683.0,
  56700.0],
 [0.0,
  0.0047975,
  0.009595,
  0.014392499999999999,
  0.01919,
  0.0239875,
  0.028784999999999998,
  0.0335825,
  0.03838,
  0.043177499999999994,
  0.047975,
  0.0527725,
  0.057569999999999996,
  0.0623675,
  0.067165,
  0.0719625,
  0.07676,
  0.08155749999999999,
  0.08635499999999999,
  0.0911525,
  0.09595,
  0.10074749999999999,
  0.105545,
  0.11034249999999998,
  0.11513999999999999,
  0.11993749999999999,
  0.124735,
  0.1295325,
  0.13433,
  0.1391275,
  0.143925,
  0.14872249999999998,
  0.15352,
  0.1583175,
  0.163114999999

In [155]:
dataPartition[0][0]

[['FIN', '2019-12-31', 0, 0],
 ['FIN', '2020-01-01', 0, 0],
 ['FIN', '2020-01-02', 0, 0],
 ['FIN', '2020-01-03', 0, 0],
 ['FIN', '2020-01-04', 0, 0],
 ['FIN', '2020-01-05', 0, 0]]

In [165]:
import json
from datetime import datetime

for i in range(len(valuePartition[0])):
    valuePartition[0][i] = datetime.utcfromtimestamp(int(valuePartition[0][i])).strftime('%Hhours %Mmins')
    
for i in range(1):
    temp = {}
    for j in range(40):
        temp[valuePartition[i][j]]= []
        for row in dataPartition[i][j]:
            if(row[2] != 0):
                temp[valuePartition[i][j]].append({"iso": str(row[0]), "date":str(row[1]), "fatality rate":str(round(float(row[3])/float(row[2]),2))})
            else:
                temp[valuePartition[i][j]].append({"iso": str(row[0]), "date":str(row[1]), "fatality rate":str(0)})
    temp[valuePartition[i][40]]= []
    with open(str(i)+'.json', 'w')as fp:
        json.dump(temp, fp)

### Corr구하기

In [157]:
# factor correlation data preparing
factorCorrData = []
for i in range(len(valuePartition)):
    corrData = []
    for j in range(40):
        corrData.append((valuePartition[i][j]+ valuePartition[i][j+1])/2.0)
    factorCorrData.append(corrData)

In [158]:
fatallityCorrData = []
for factor in dataPartition:
    corrData = []
    for index in range(40):
        totalDeath = 0
        totalCase = 0
        for row in factor[index]:
            totalCase += row[2]
            totalDeath += row[3]
        if(totalDeath == 0.0):
            corrData.append(0)
        else:
            corrData.append(float(totalCase)/float(totalDeath))
    fatallityCorrData.append(corrData)
        

In [159]:
for i in range(7):
    correlationDayTime = [fatallityCorrData[i], factorCorrData[i]]

    df = pd.DataFrame(correlationDayTime).T
    corr = df.corr(method= 'pearson')
    print(corr[1][0])

0.47915409252921176
-0.4203693846391811
0.28995141724288764
-0.22713404453050678
0.21645761167813923
0.1931454958494522
-0.2816937076549629
